In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx


### 1. 数据处理

In [1]:
import pandas as pd

# 读取xlsx文件
df = pd.read_excel('/kaggle/input/jd_comment_with_label/jd_comment_data.xlsx')

# 提取指定字段（示例字段名为'字段1','字段2','字段3'）
selected_columns = ['评价内容(content)', '评分（总分5分）(score)']
df_selected = df[selected_columns]
print(df_selected[1:10])

# 保存为csv文件，重新指定字段的名字 content label（不包含索引）
df_selected.columns = ['content', 'label']
# 如果label 小于3 则为0， 大于3 则为1
df_selected['label'] = df_selected['label'].apply(lambda x: 0 if x <= 3 else 1)
print(df_selected[1:10])
# 保存为csv文件，重新指定字段的名字 content label（不包含索引）
df_selected.to_csv('/kaggle/working/jd-comments.csv', index=False)
print("数据处理结束")

  评价内容(content)  评分（总分5分）(score)
1    此用户未填写评价内容                5
2    此用户未填写评价内容                3
3    此用户未填写评价内容                5
4    此用户未填写评价内容                5
5    此用户未填写评价内容                4
6    此用户未填写评价内容                5
7    此用户未填写评价内容                5
8    此用户未填写评价内容                5
9    此用户未填写评价内容                5
      content  label
1  此用户未填写评价内容      1
2  此用户未填写评价内容      0
3  此用户未填写评价内容      1
4  此用户未填写评价内容      1
5  此用户未填写评价内容      1
6  此用户未填写评价内容      1
7  此用户未填写评价内容      1
8  此用户未填写评价内容      1
9  此用户未填写评价内容      1
数据处理结束


/tmp/ipykernel_35/3346069408.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['label'] = df_selected['label'].apply(lambda x: 0 if x <= 3 else 1)


### 2.定义数据加载器

In [8]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

# 定义数据集类 继承自torch.utils.data.Dataset
import torch
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, contents, labels):
        self.contents = contents  # 文本数据
        self.labels = labels      # 标签数据
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        return {
            'content': self.contents.iloc[idx],
            'label': torch.tensor(self.labels.iloc[idx])
        }
def create_dataloader(contents, labels, tokenizer, batch_size=32, shuffle=True):
    # 创建数据集
    dataset = TextDataset(contents, labels)
    
    def collate_fn(batch):
        # 自定义collate函数，用于处理不同长度的文本
        contents = [item['content'] for item in batch]
        labels = [item['label'] for item in batch]
        # 使用tokenizer处理文本
        inputs = tokenizer(
            contents, # 文本列表
            padding=True, # 填充到最大长度
            truncation=True, # 截断到最大长度
            max_length=512, # embeding_dim 最大长度
            return_tensors="pt" # 返回PyTorch张量
        )
        return {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'labels': torch.tensor(labels, dtype=torch.long)
        }
    # 创建数据加载器
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )

### 3.加载tokenizer及model

In [10]:
from transformers import AutoTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
# 加载预训练的BERT模型和分词器
model_name = 'google-bert/bert-base-chinese' # 预训练模型名称
tokenizer = AutoTokenizer.from_pretrained(model_name) # 加载分词器
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2) # 加载BERT模型，指定标签数量为2（好评和差评）
model.config.problem_type = "single_label_classification"
# model = model.float()
print(model)
# tensorboard 记录器
writer = SummaryWriter("/kaggle/working/logs")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### 3.加载数据并划分数据集

In [11]:
# 读取CSV文件
df = pd.read_csv("/kaggle/working/jd-comments.csv")
print(df[0:5])
# 划分训练集和测试集
train_contents, test_contents, train_labels, test_labels = train_test_split(
    df['content'], df['label'], test_size=0.2, random_state=42
) 
print(train_contents.shape)
print(train_contents[1:10])
print(train_labels[1:10])

      content  label
0  此用户未填写评价内容      1
1  此用户未填写评价内容      1
2  此用户未填写评价内容      0
3  此用户未填写评价内容      1
4  此用户未填写评价内容      1
(57454,)
52                    此用户未填写评价内容
16168          质量超级好啊 穿上哈还好看 太值了
25529            用上了ssd，软件开启速度飞快
38045    老公说以前上厕所没信号，现在都是满格！很不错呢
29141                 此用户未填写评价内容
11852                 此用户未填写评价内容
42073                东西不错！也挺牢固的！
14969                         不错
32570                      可以，好用
Name: content, dtype: object
52       1
16168    1
25529    1
38045    1
29141    1
11852    1
42073    1
14969    1
32570    1
Name: label, dtype: int64


In [12]:
# 创建训练集和测试集的数据加载器
train_dataloader = create_dataloader(train_contents, train_labels, tokenizer, batch_size=32)
test_dataloader = create_dataloader(test_contents, test_labels, tokenizer, batch_size=32)

### 开始训练并保存模型

In [14]:
# 检查是否有可用的GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # 使用GPU
else:
    device = torch.device("cpu")  # 使用CPU

# 定义优化器和损失函数
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5) # 优化器，使用AdamW
# 定义损失函数
criterion = torch.nn.CrossEntropyLoss() # 损失函数，使用交叉熵损失  
# 训练模型
num_epochs = 3 # 训练轮数
# 模型加载到GPU
model.to(device)
for epoch in range(num_epochs):
    model.train() # 设置模型为训练模式
    total_loss = 0 # 初始化总损失
    for batch in train_dataloader: # 遍历训练集数据加载器
        batch['labels'] = batch['labels'].long()
        # 打印数据类型
        #print(f"Batch data types: {batch['input_ids'].dtype}, {batch['attention_mask'].dtype}, {batch['labels'].dtype}")
        
        # 将数据加载到GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        # 清零梯度
        optimizer.zero_grad()
        # 前向传播
        outputs = model(**batch) # 调用模型进行前向传播，传入输入数据
        # 计算损失
        loss = outputs.loss # 从模型输出中获取损失值
        # 反向传播
        loss.backward() # 反向传播计算梯度
        # 更新参数
        optimizer.step() # 更新模型参数 
        # 打印损失值
        # print(f"Batch loss: {loss.item()}")
        # 累加损失值
        total_loss += loss.item() * batch['input_ids'].size(0) # 累加损失值，乘以批次大小
        # 记录损失值到tensorboard
        if writer is not None:
            if not torch.isfinite(loss):
                print("Loss is infinite or NaN, skipping this batch.")
                continue
            writer.add_scalar("Loss/train", loss.item(), epoch * len(train_dataloader))
    # 计算平均损失
    avg_loss = total_loss / len(train_dataloader.dataset) # 计算平均损失
    # 打印结果
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}") # 打印当前轮数和平均损失
    # 保存模型
    torch.save(model.state_dict(), f"/kaggle/working/model_epoch_{epoch+1}.pth") # 保存模型参数，文件名为"model_epoch_{epoch+1}.pth"
    print(f"Model saved as model_epoch_{epoch+1}.pth") # 打印保存模型的信息

Epoch 1/3, Loss: 0.0879
Model saved as model_epoch_1.pth
Epoch 2/3, Loss: 0.0742
Model saved as model_epoch_2.pth
Epoch 3/3, Loss: 0.0637
Model saved as model_epoch_3.pth


In [16]:
# 验证模型 
model.to(device)
model.eval() # 设置模型为评估模式
total_loss = 0 # 初始化总损失
total_correct = 0 # 初始化正确预测数量
total_samples = 0 # 初始化样本数量
with torch.no_grad(): # 禁用梯度计算，减少内存消耗
    for batch in test_dataloader: # 遍历测试集数据加载器
         # 将数据加载到GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        # 前向传播  
        outputs = model(**batch) # 调用模型进行前向传播，传入输入数据
        # 计算损失
        loss = outputs.loss # 从模型输出中获取损失值
        total_loss += loss.item() * batch['input_ids'].size(0) # 累加损失值，乘以批次大小  
        # 计算预测结果
        logits = outputs.logits # 从模型输出中获取预测结果
        predictions = torch.argmax(logits, dim=-1) # 获取预测结果的类别索引 
        # 计算准确率
        total_correct += (predictions == batch['labels']).sum().item() # 累加正确预测数量
        total_samples += batch['labels'].size(0) # 累加样本数量 
# 计算平均损失和准确率 
avg_loss = total_loss / total_samples # 计算平均损失
accuracy = total_correct / total_samples # 计算准确率
# 打印结果
print(f"Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}") # 打印当前轮数、平均损失和准确率

Loss: 0.0792, Accuracy: 0.9754


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 加载微调后的模型
model_path = "/kaggle/working/model_epoch_1.pth"
model_name = "google-bert/bert-base-chinese"
label_map = {0: "负面", 1: "正面"}
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.load_state_dict(torch.load(model_path))
model.eval()

# 推理函数
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", 
                      padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        pred_label = torch.argmax(probs).item()
    return {
        "label": label_map[pred_label],
        "confidence": probs[0][pred_label].item()
    }

h:\anaconda\miniconda3\envs\study-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [29]:
# 推理示例
text = "一般般，一分钱一分货吧"
result = predict(text)
print("lable:", result['label'])
print("lable:", result['confidence'])

lable: 正面
lable: 0.7131579518318176
